In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import log_loss
import theano as th

/Users/danielg/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [21]:
from sklearn.decomposition import TruncatedSVD

In [3]:
th.version.full_version

'0.7.0.dev-dfb2730348d05f6aadd116ce492e836a4c0ba6d6'

# Data preparation

## read

In [9]:
submission_sample = pd.read_csv("data/sample_submission.csv")
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

## clean and create features

In [10]:
# create id
train['id'] = train.index.values
test['id'] = test.index.values

In [11]:
train.apply(lambda x: len(x.unique()))

TripType                     38
VisitNumber               95674
Weekday                       7
Upc                       97715
ScanCount                    39
DepartmentDescription        69
FinelineNumber             5196
id                       647054
dtype: int64

In [12]:
# could create variable with number of items per visit???
# will probably be quite useful??
    # will be able to create other variables like: departmenttype flat, etc - so quite a lot of other variables

unique_trip_items_train = train.groupby(['VisitNumber']).agg({'id': len})
unique_trip_items_train.columns = ['unique_trip_items']

unique_trip_items_test = test.groupby(['VisitNumber']).agg({'id': len})
unique_trip_items_test.columns = ['unique_trip_items']

# also total number by the scancounts...
trip_scancount_total_train = train.groupby(['VisitNumber']).agg({'ScanCount': sum})
trip_scancount_total_train.columns = ['ScanCount_total']

trip_scancount_total_test = test.groupby(['VisitNumber']).agg({'ScanCount': sum})
trip_scancount_total_test.columns = ['ScanCount_total']

In [13]:
#
# missing value imputation
#

# will set missing FinelineNumber to -1
train.FinelineNumber = train.FinelineNumber.fillna(-1)
test.FinelineNumber = test.FinelineNumber.fillna(-1)
# will set missing Upc to 0
train.Upc = train.Upc.fillna(0)
test.Upc = test.Upc.fillna(0)
# will set missing DepartmentDescription to Missing
train.DepartmentDescription = train.DepartmentDescription.fillna("missing")
test.DepartmentDescription = test.DepartmentDescription.fillna("missing")

In [14]:
train_grouped = train.groupby("VisitNumber", as_index=False).agg({'Weekday': lambda x: x.unique(),
                               'id': len,
                              'ScanCount': sum,
                             'TripType': lambda x: x.unique()})

In [15]:
test_grouped = test.groupby("VisitNumber", as_index=False).agg({'Weekday': lambda x: x.unique(),
                               'id': len,
                              'ScanCount': sum})

In [16]:
X_train = train_grouped.drop("TripType",axis=1)
y_train = train_grouped.TripType

In [17]:
# integrate weekday data
days = X_train.Weekday.unique()
for day in days:
    X_train[str(day)] = (X_train.Weekday == str(day)).astype(int)
X_train = X_train.drop("Weekday",axis=1)

X_test = test_grouped
days = X_test.Weekday.unique()
for day in days:
    X_test[str(day)] = (X_test.Weekday == str(day)).astype(int)
X_test = X_test.drop("Weekday",axis=1)

In [18]:
# integrate department data

department_crosstab = pd.crosstab(train.VisitNumber, train.DepartmentDescription)
# sort index because the order of the columns needs to be identical in test and train
X_train = X_train.merge(department_crosstab.sort_index(axis=1), left_on='VisitNumber', right_index=True)
department_crosstab_test = pd.crosstab(test.VisitNumber, test.DepartmentDescription)
# how to get missing columns that are IN the training but NOT test
# 
missing_col_test = department_crosstab.columns[~department_crosstab.columns.isin(department_crosstab_test.columns)].values[0]
# how to get missing columns that IN the test but NOT training
department_crosstab_test.columns[~department_crosstab_test.columns.isin(department_crosstab.columns)].values
# need to add the columns
department_crosstab_test[missing_col_test] = 0
X_test = X_test.merge(department_crosstab_test.sort_index(axis=1), left_on='VisitNumber', right_index=True)
# need to make sure that all of the columns match
all(X_train.columns == X_test.columns)

True

#### create train, validation indices...

In [25]:
y_train_dummies = pd.get_dummies(y_train).as_matrix()
sss = StratifiedShuffleSplit(y_train_dummies,1, test_size=0.25, random_state=0)
for train_index, test_index in sss:
    #X_train_, X_val = X_train_mat[train_index], X_train_mat[test_index]
    y_train_, y_val = y_train_dummies[train_index], y_train_dummies[test_index]
    

In [19]:
# integrate fineline data

# want to do the same for FineLinenumber
fineline_train_crosstab = pd.crosstab(train.VisitNumber, train.FinelineNumber)
fineline_test_crosstab = pd.crosstab(test.VisitNumber, test.FinelineNumber)

In [20]:
fineline_not_in_train = fineline_test_crosstab.columns[~fineline_test_crosstab.columns.isin(fineline_train_crosstab.columns)].values
fineline_test_crosstab = fineline_test_crosstab.drop(fineline_not_in_train,axis=1)
# want to drop train columns that are not contained in test
# need a 'not in test' variable
fineline_not_in_test = fineline_train_crosstab.columns[~fineline_train_crosstab.columns.isin(fineline_test_crosstab.columns)].values
fineline_train_crosstab = fineline_train_crosstab.drop(fineline_not_in_test,axis=1)
# need to make sure that all of these match
all(fineline_test_crosstab.columns == fineline_train_crosstab.columns)

True

### split fineline into train and val

In [ ]:
for train_index, test_index in sss:
    fineline_train_, fineline_val = fineline_train_crosstab.as_matrix()[train_index], fineline_train_crosstab.as_matrix()[test_index]

In [ ]:
## want to apply truncated SVD to this
svd = TruncatedSVD(n_components=5, random_state=42)
svd.fit(X)


In [72]:
# switching this over to test sums - to emphasise this...??

# have a column mismatch need to fix this ....‚
fineline_ordered_sums = fineline_train_crosstab.sum(0).sort_values(ascending=False)
finelines_incl = fineline_ordered_sums.index
# .iloc[0:1000]

In [73]:
fineline_train_crosstab_r = fineline_train_crosstab.loc[:,finelines_incl]
fineline_test_crosstab_r = fineline_test_crosstab.loc[:,finelines_incl]


In [ ]:
# merge into fineline one hot encoded into X_train and X_test
X_train = X_train.merge(fineline_train_crosstab_r, left_on="VisitNumber", right_index=True)
X_test = X_test.merge(fineline_test_crosstab_r, left_on="VisitNumber", right_index=True)

## create train and test matrices

In [ ]:
y_train_dummies = pd.get_dummies(y_train).as_matrix()
X_train = X_train.drop(['VisitNumber', 'id'], axis=1)
X_test = X_test.drop(['VisitNumber', 'id'], axis=1)

In [ ]:
# get rid of columns with a low number of observations....

to_keep = X_train.sum()[X_train.sum() > 100].index
X_train = X_train[to_keep]
X_test = X_test[to_keep]

In [ ]:
X_train_mat = X_train.as_matrix()
X_test_mat = X_test.as_matrix()

In [ ]:
sss = StratifiedShuffleSplit(y_train,1, test_size=0.25, random_state=0)

In [ ]:
for train_index, test_index in sss:
    X_train_, X_val = X_train_mat[train_index], X_train_mat[test_index]
    y_train_, y_val = y_train_dummies[train_index], y_train_dummies[test_index]

In [ ]:
X_train_.shape

In [ ]:
#normalisation

# col means
# col std
col_means = X_train_.mean(0)
col_std = X_train_.std(0)

In [ ]:
# apply to train
X_train_ = (X_train_-col_means)/col_std

# apply to val
X_val = (X_val-col_means)/col_std

# apply to test
X_test_mat = (X_test_mat-col_means)/col_std

# Model

In [54]:
model = Sequential()

model.add(Dense(256, input_dim=X_train.shape[1], init='uniform'))
model.add(Activation('tanh'))
model.add(Dropout(0.2)) 
model.add(Dense(256, init='uniform'))
model.add(Activation('tanh'))
model.add(Dropout(0.2)) 
model.add(Dense(y_train_.shape[1], init='uniform'))
model.add(Activation('softmax'))

In [49]:
th.config.compute_test_value = "off"

In [55]:
model.compile(loss='categorical_crossentropy', optimizer="RMSprop")

In [60]:
model.fit(X_train_, y_train_, nb_epoch=10, batch_size=100)

Epoch 1/100
71753/71753 [==============================] - 15s - loss: 0.7739    
Epoch 2/100
71753/71753 [==============================] - 15s - loss: 0.7586    
Epoch 3/100
71753/71753 [==============================] - 16s - loss: 0.7545    
Epoch 4/100
71753/71753 [==============================] - 16s - loss: 0.7454    
Epoch 5/100
71753/71753 [==============================] - 17s - loss: 0.7346    
Epoch 6/100
71753/71753 [==============================] - 17s - loss: 0.7234    
Epoch 7/100
71753/71753 [==============================] - 17s - loss: 0.7141    
Epoch 8/100
71753/71753 [==============================] - 17s - loss: 0.7121    
Epoch 9/100
71753/71753 [==============================] - 17s - loss: 0.7004    
Epoch 10/100
71753/71753 [==============================] - 17s - loss: 0.6962    
Epoch 11/100
71753/71753 [==============================] - 17s - loss: 0.6910    
Epoch 12/100
71753/71753 [==============================] - 17s - loss: 0.6788    
Epoch 13/100


In [61]:
predicted = model.predict(X_val)
log_loss(y_true=y_val, y_pred=predicted)

0.97176919177495635

In [ ]:
# if making final prediction
predicted = model.predict(X_test.as_matrix())
predicted = pd.DataFrame(predicted)
predicted = pd.concat([submission_sample.VisitNumber, predicted],axis=1)
predicted.columns = submission_sample.columns

In [ ]:
predicted.to_csv('submission_test.csv', index=False)